<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Все-задания-нужно-выполнить-только-с-помощью-sql" data-toc-modified-id="Все-задания-нужно-выполнить-только-с-помощью-sql-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Все задания нужно выполнить только с помощью sql</a></span></li></ul></div>

In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(':memory:')

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [4]:
data.to_sql('boston', con, index=False, if_exists='replace')

In [5]:
a = pd.read_sql(
    '''
    SELECT * FROM boston
    limit 5
    ''',
    con,
)
a

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [6]:
tasks = 0

### Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [7]:
tasks+=1
pd.read_sql(
    '''
SELECT COUNT(*) cnt, 'CRIM'  variable FROM boston WHERE CRIM  IS NULL
    UNION ALL
SELECT COUNT(*) cnt, 'ZN'    variable FROM boston WHERE ZN    IS NULL
    UNION ALL
SELECT COUNT(*) cnt, 'INDUS' variable FROM boston WHERE INDUS IS NULL
    UNION ALL
SELECT COUNT(*) cnt, 'CHAS'  variable FROM boston WHERE CHAS  IS NULL
    UNION ALL
SELECT COUNT(*) cnt, 'NOX'   variable FROM boston WHERE NOX   IS NULL
    ''',
    con,
)


,cnt,variable
0,0,CRIM
1,0,ZN
2,0,INDUS
3,0,CHAS
4,0,NOX


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [8]:
tasks+=1

pd.read_sql(
    '''
SELECT COUNT(*) cnt, 'CRIM'  variable FROM (SELECT DISTINCT CRIM  FROM boston)
    UNION ALL
SELECT COUNT(*) cnt, 'ZN'    variable FROM (SELECT DISTINCT ZN    FROM boston)
    UNION ALL
SELECT COUNT(*) cnt, 'INDUS' variable FROM (SELECT DISTINCT INDUS FROM boston)
    UNION ALL
SELECT COUNT(*) cnt, 'CHAS'  variable FROM (SELECT DISTINCT CHAS  FROM boston)
    UNION ALL
SELECT COUNT(*) cnt, 'NOX'   variable FROM (SELECT DISTINCT NOX   FROM boston)
    ''',
    con,
)

,cnt,variable
0,504,CRIM
1,26,ZN
2,76,INDUS
3,2,CHAS
4,81,NOX


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [9]:
tasks+=1

pd.read_sql(
    ''' WITH step_1 AS (
                SELECT 
                  'CRIM' name, 
                   CRIM value 
                FROM boston
                UNION ALL
                
                SELECT 
                  'ZN' name, 
                   ZN value 
                FROM boston
                UNION ALL
                
                SELECT 
                  'INDUS' name, 
                  INDUS value 
                FROM boston
                UNION ALL
                
                SELECT 
                  'CHAS' name, 
                   CHAS value 
                FROM boston
                UNION ALL
                
                SELECT 
                  'NOX' name, 
                   NOX value 
                FROM boston ),
                
            step_2 AS ( SELECT 
                          name, 
                          value,
                          ROW_NUMBER() OVER(PARTITION BY name ORDER BY value) as rn,
                          MIN(value) OVER(PARTITION BY name) min_value,
                          COUNT(value) OVER(PARTITION BY name) as cnt
                        FROM step_1 )
            SELECT name 
            FROM (  SELECT 
                      name, 
                      value as md_value, 
                      min_value
                    FROM step_2
                    WHERE rn = cnt/2 )
            WHERE md_value = min_value
    ''',
    con
)

,name
0,CHAS
1,ZN


Вывод: минимальные значения выявлены у двух столбцов. 

4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [10]:
tasks+=1
pd.read_sql('''
WITH high_price AS ( SELECT avg(rm) AS avg_high FROM boston GROUP BY medv ORDER BY medv DESC LIMIT 1),
       limit_25 AS ( SELECT 'limit 25'  category, avg(rm) rm_avg, avg_high rm_avg_h, avg_high-avg(rm) diff FROM ( SELECT rm, medv FROM boston ORDER BY medv ASC LIMIT 25),  high_price),
       limit_50 AS ( SELECT 'limit 50'  category, avg(rm) rm_avg, avg_high rm_avg_h, avg_high-avg(rm) diff FROM ( SELECT rm, medv FROM boston ORDER BY medv ASC LIMIT 50),  high_price),
      limit_100 AS ( SELECT 'limit 100' category, avg(rm) rm_avg, avg_high rm_avg_h, avg_high-avg(rm) diff FROM ( SELECT rm, medv FROM boston ORDER BY medv ASC LIMIT 100), high_price), 
      limit_200 AS ( SELECT 'limit 200' category, avg(rm) rm_avg, avg_high rm_avg_h, avg_high-avg(rm) diff FROM ( SELECT rm, medv FROM boston ORDER BY medv ASC LIMIT 200), high_price),
      limit_300 AS ( SELECT 'limit 300' category, avg(rm) rm_avg, avg_high rm_avg_h, avg_high-avg(rm) diff FROM ( SELECT rm, medv FROM boston ORDER BY medv ASC LIMIT 300), high_price)
    
    SELECT * FROM limit_25
    UNION ALL
    SELECT * FROM limit_50
    UNION ALL
    SELECT * FROM limit_100
    UNION ALL
    SELECT * FROM limit_200
    UNION ALL
    SELECT * FROM limit_300
    ''',
    con,
)

,category,rm_avg,rm_avg_h,diff
0,limit 25,5.747840,7.484,1.736160
1,limit 50,5.753240,7.484,1.730760
2,limit 100,5.887120,7.484,1.596880
3,limit 200,5.911705,7.484,1.572295
4,limit 300,5.972227,7.484,1.511773


In [11]:
# Вывод: Однозначного ответа согласно рассчетрной части нет. Однако даже логически если подумать больше площать больше стоимость. 

5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех значений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [12]:
tasks+=1
print('Вывод:  в дорогих домах низкая численность населения с низким статусом')
pd.read_sql(
    """
    SELECT 
      rank,
      LSTAT,
      MEDV  
    FROM ( SELECT 
             MEDV,
             LSTAT,
             RANK() OVER(ORDER BY LSTAT) rank,
             MAX(MEDV) over() max_
           FROM boston )
    WHERE MEDV = max_
    """,
    con,
)


Вывод:  в дорогих домах низкая численность населения с низким статусом


,rank,LSTAT,MEDV
0,1,1.73,50.0
1,2,1.92,50.0
2,6,2.88,50.0
3,8,2.96,50.0
4,9,2.97,50.0
5,15,3.16,50.0
6,17,3.26,50.0
7,18,3.32,50.0
8,26,3.70,50.0
9,27,3.73,50.0


6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [13]:
tasks+=1
pd.read_sql(
    """
    SELECT DISTINCT 
      (CASE
        WHEN CHAS = 1 THEN 'Граничит'
        ELSE 'Не граничит'
      END) CHAS,
      round(avg(MEDV) over(PARTITION BY CHAS)) avg_MEDV
    FROM boston
    """,
    con,
)

,CHAS,avg_MEDV
0,Не граничит,22.0
1,Граничит,28.0


7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [14]:
tasks+=1
pd.read_sql(
    """
    SELECT col_name FROM 
    (SELECT 
        'CRIM' AS col_name, 
        (SELECT avg(crim) FROM boston WHERE chas=1)>avg(crim) AS flag
    FROM boston
    UNION
    SELECT 
        'ZN' AS col_name, 
        (SELECT avg(ZN) FROM boston WHERE chas=1)>avg(ZN) AS flag
    FROM boston
    UNION
    SELECT 
        'INDUS' AS col_name, 
        (SELECT avg(INDUS) FROM boston WHERE chas=1)>avg(INDUS) AS flag
    FROM boston
    
    
    
    UNION
    SELECT 
        'CHAS' AS col_name, 
        (SELECT avg(CHAS) FROM boston WHERE chas=1)>avg(CHAS) AS flag
    FROM boston   
    
    UNION
    SELECT 
        'NOX' AS col_name, 
        (SELECT avg(NOX) FROM boston WHERE chas=1)>avg(NOX) AS flag
    FROM boston
    )
    WHERE flag = 1
    """,
    con,
)
# Рядом с рекой много промышленных объектов

,col_name
0,CHAS
1,INDUS
2,NOX


8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [15]:
tasks+=1
print('Есть связь между долей застройки и концентрацией азота')
pd.read_sql( """
       WITH step_1 AS (SELECT 
                         INDUS, 
                         NTILE(10) OVER (ORDER BY INDUS) percentile
                       FROM boston),
                   
        step_2 AS (SELECT 
                  NOX, 
                  NTILE(10) OVER (ORDER BY NOX) percentile
                FROM boston)    
select 
  a.percentile * 10 as percentile ,
  a.indus,
  b.nox 
from (SELECT 
        percentile, 
        MAX(indus) as indus
      FROM step_1
      GROUP BY percentile) a
      LEFT JOIN (SELECT 
                   percentile, 
                   MAX(nox) as nox
                 FROM step_2
                 GROUP BY percentile) b ON a.percentile = b.percentile 
    """,
    con
)

Есть связь между долей застройки и концентрацией азота


,percentile,indus,nox
0,10,2.89,0.426
1,20,4.39,0.442
2,30,5.96,0.472
3,40,7.38,0.507
4,50,9.69,0.538
5,60,13.89,0.580
6,70,18.10,0.609
7,80,18.10,0.671
8,90,19.58,0.713
9,100,27.74,0.871


In [16]:
if tasks==8:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')

Выполнены все задания 🙂
